In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import os
from os.path import join as oj
from sklearn.feature_extraction.image import extract_patches_2d
from sklearn.linear_model import LinearRegression, LogisticRegression, RidgeCV
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import cross_validate, train_test_split
from sklearn import metrics
import numpy as np
from collections import Counter
from sklearn.datasets import make_classification
from torch import nn
import torch.nn.functional as F
import torch
from copy import deepcopy
from sklearn import metrics
plt.style.use('dark_background')
import mat4py
import pandas as pd
import data_tracks
from skorch.callbacks import Checkpoint, TrainEndCheckpoint
from skorch import NeuralNetRegressor, NeuralNetClassifier
import models
from imblearn.over_sampling import RandomOverSampler
from statsmodels import robust
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVC, SVR
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.model_selection import KFold
import pickle as pkl
from tqdm import tqdm
from sklearn.metrics import r2_score
import train_reg
#df = data_tracks.get_tracks() # note: different Xs can be different shapes
#df = data_tracks.remove_invalid_tracks(df)
#df = data_tracks.preprocess(df)
#df = data_tracks.add_outcome(df)
#n = df.shape[0]

Using TensorFlow backend.
/usr/local/linux/anaconda3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/linux/anaconda3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/linux/anaconda3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/linux/anaconda3.7/lib/pytho

In [6]:
df = data_tracks.get_data()
outcome_def = 'Y_peak_time_frac_diff'
df[outcome_def] = df['Y_peak_time_frac'] - df['X_peak_time_frac']
out_dir = f'results/regression/{outcome_def}'
os.makedirs(out_dir, exist_ok=True)
feat_names = data_tracks.get_feature_names(df)
feat_names = [x for x in feat_names 
              if not x.startswith('sc_') 
              and not x.startswith('nmf_')
              and not x in ['center_max', 'left_max', 'right_max', 'up_max', 'down_max',
                           'X_max_around_Y_peak', 'X_max_after_Y_peak', 'X_max_diff']
              and not x.startswith('pc_')
#               and not 'slope' in x
             ]

In [7]:
for model_type in tqdm(['linear', 'gb', 'rf', 'svm', 'mlp2', 'dt']):
    out_name = f'{model_type}'
                    #print(out_name)
    train_reg.train_reg(df, feat_names=feat_names, model_type=model_type, 
                 outcome_def=outcome_def,
                 out_name=f'{out_dir}/{out_name}.pkl') 

 17%|█▋        | 1/6 [00:00<00:01,  4.58it/s]

[0.2945880925242653, 0.20127574441155272, 0.3278371925355047, 0.3280825877168695, 0.1852003281939979]


 33%|███▎      | 2/6 [00:02<00:02,  1.41it/s]

[0.26973759706574063, 0.20305426129920803, 0.3380544947402896, 0.30169884628075694, 0.22414742265276555]


 50%|█████     | 3/6 [00:14<00:12,  4.24s/it]

...

[0.0276368851706853, 0.10487650995324171, 0.06163737918878032, 0.06351410767000099, 0.11992407727168021]


 83%|████████▎ | 5/6 [00:22<00:04,  4.43s/it]

[-13196.854495265366, -199139.11197373163, -346212.5256369297, -139816.4304833864, -384009.4806216045]


100%|██████████| 6/6 [00:22<00:00,  3.17s/it]

[-0.3871093430273531, -0.7056751389593561, -0.2610547624959758, -0.42901063469465406, -0.42252814259492744]


In [8]:
results = train_reg.load_results(out_dir)
r = results
r = r[[k for k in r if not 'std' in k]]
r = r[[k for k in r if not '_f' in k]]
# r = r[r.index.str.contains('ros')] # only use random sampling
r = r.sort_values(by=['r2'], ascending=False)
# r.style.background_gradient(cmap='viridis', axis=None) # all values on same cmap
r

,cv_accuracy_by_cell,r2
model_type,,
rf,"[0.2698458040429347, 0.24394621933804617, 0.35...",0.281
linear,"[0.2945880925242653, 0.20127574441155272, 0.32...",0.267
gb,"[0.26973759706574063, 0.20305426129920803, 0.3...",0.265
svm,"[0.0276368851706853, 0.10487650995324171, 0.06...",0.076
dt,"[-0.3871093430273531, -0.7056751389593561, -0....",-0.449
mlp2,"[-13196.854495265366, -199139.11197373163, -34...",-206259.381
